# Predviđanje moždanog udara

### Uvod u problematiku
Prema podacima svjetske zdrastvene organizacije (eng. WHO), moždani udar je drugi najučestaliji uzročnik smrti, odgovoran za čak 11% svih smrtnih slučajeva čovječanstva. Iako ulogu svakako igraju genetske predispozicije, najbitniji faktor je način života. Ovaj esej posvećen je analizi i predviđanju faktora koji utječu na mogućnost dobivanja moždanog udara na osnovi podataka preuzetih s "https://www.kaggle.com".
### Učitavanje podataka

In [19]:
import pandas as pd
import numpy as np
podaci = pd.read_csv('podaci.csv', encoding='utf-8')
print(podaci.shape)
print(podaci.info())

(5110, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB
None


In [20]:
podaci.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


### Dokumentacija 

Uz jedinstvenu oznaku pacijenta u prvom stupcu i indikatorske varijable koja pokazuje je li pacijent imao srčani udar ili ne u posljednjem stupcu, sve ostale stupce čine varijable za koje se medicinski vjeruje da igraju ulogu u dobivanju srčanog udara. Opis vrijednosti u pojedinim stupcima dane su u nastavku.
1) **id**: jedinstvena oznaka pacijenta <br>
2) **gender**: spol, može biti "Male", "Female" or "Other" <br>
3) **age**: starost pacijenta <br>
4) **hypertension**: 0 ako pacijent nije imao hipertenziju, 1 ako jest <br>
5) **heart_disease**: 0 ako pacijent nije imao nikakvu srčanu bolest, 1 ako jest <br>
6) **ever_married**: bračni status pacijenta, može biti "No" or "Yes" <br>
7) **work_type**: tip posla, može biti "children", "Govt_jov", "Never_worked", "Private" or "Self-employed" <br>
8) **Residence_type**: mjesto stanovanja, može biti "Rural" or "Urban" <br>
9) **avg_glucose_level**: prosječna razina glukoze u krvi  <br>
10) **bmi**: indeks tjelesne mase <br>
11) **smoking_status**: može biti "formerly smoked", "never smoked", "smokes" or "Unknown"* <br>
12) **stroke**: 1 ako je pacijent imao moždani udar ili 0 ako nije <br>
*"Unknown" kod smoking_status znači da je ta informacija nedostupna za pacijenta*

### Upoznavanje s podacima

Najprije uočimo da nam je stupac "id" potpuno nepotreban. Ono što nam je često bitno kod analiza i općenito u statistici jest da imamo "dobar" uzorak. No, najprije uočimo da nam je stupac "id" potpuno nepotreban. Za one varijable koje su kontinuirane, odnosno neprekidne, možemo izračunati standarnu desktiptivnu statistiku.

In [21]:
podaci = podaci.drop('id', axis=1)

In [24]:
podaci[['age','avg_glucose_level', 'bmi']].describe()

,age,avg_glucose_level,bmi
count,5110.000000,5110.000000,4909.000000
mean,43.226614,106.147677,28.893237
std,22.612647,45.283560,7.854067
min,0.080000,55.120000,10.300000
25%,25.000000,77.245000,23.500000
50%,45.000000,91.885000,28.100000
75%,61.000000,114.090000,33.100000
max,82.000000,271.740000,97.600000
